In step three, the XLRM specification is used to build a diverse ensemble of possible future states ofthe world (SOWs), or scenarios, that span the uncertainty space specified in the first step. The set offuture SOWs are used to determine how the defined policies may react to a wide variety of possiblefutures. This stage generally involves the use of software to both build the ensemble of scenariosandto apply that ensembleto the set ofpolicy options to build adata set that evaluatesthe potentialeffectiveness of a policy given the XLRM problem definition.

# Import

In [1]:
#   import packages
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
import networkx as nx
import pickle
import functools

from ema_workbench import (
    Model,
    Policy,
    Scenario,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    util,
    ScalarOutcome,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress
from ema_workbench.em_framework import parameters_from_csv
from ema_workbench.em_framework.evaluators import BaseEvaluator
from ema_workbench.analysis import parcoords
from ema_workbench.analysis import prim
from ema_workbench import Samplers

Then, the model is loaded, and logging is set.

In [2]:
dike_model, planning_steps = get_model_for_problem_formulation(5)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

First, the results of the policy discovery is loaded.

In [3]:
pareto_policies = pd.read_pickle(r'../generated_datasets/initial_Pareto_policies.pkl')

Then, the policies from the analysis are loaded into the model.

In [4]:
policies = []
for row_number in range(pareto_policies.shape[0]):
    policies.append(
        Policy(name=row_number, **pareto_policies.iloc[row_number, :-5].to_dict())
    )

General settings of the model are set.

In [5]:
n_scenarios = int(10_000)

Then, run the model with the robust policies as input, and generate results over a lot of scenarios

In [ ]:
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=n_scenarios,
                                            policies=policies,
                                            uncertainty_sampling=Samplers.LHS
                                            )

with open(r'../generated_datasets/policy_uncertainty_test.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 10000 scenarios * 45 policies * 1 model(s) = 450000 experiments
  0%|                                  | 163/450000 [00:16<10:34:08, 11.82it/s]

The results are then loaded in for analysis

In [ ]:
scenario_results = pd.read_pickle(r'../generated_datasets/policy_uncertainty_test.pkl')
experiments, outcomes = scenario_results

Make a dataframe column for every timestep of the results.

In [ ]:
policies_all_scenarios = pd.DataFrame()
for key, value in outcomes.items():
    temp_df = pd.DataFrame(value, columns=[key + ' '+ str(x) for x in range(len(value[0]))])
    policies_all_scenarios = pd.concat([policies_all_scenarios, temp_df], axis=1)

Add experiments input to the outputs for analysis

In [ ]:
experiments_and_results = pd.concat([experiments,policies_all_scenarios], axis=1)

with open(r'../generated_datasets/policy_uncertainty_experiments_results.pkl', 'wb') as file_pi:
    pickle.dump(experiments_and_results, file_pi)

Combine the variables of different dikes and times

In [ ]:
def combine_columns(dataframe, name):
    combined_columns = []
    for x in dataframe.columns:
        if name in x:
            combined_columns.append(x)

    dataframe['Total ' + str(name)] = dataframe[combined_columns].sum(axis=1)

In [ ]:
combine_columns(policies_all_scenarios, 'Expected Annual Damage')
combine_columns(policies_all_scenarios, 'Dike Investment Costs')
combine_columns(policies_all_scenarios, 'Expected Number of Deaths')
combine_columns(policies_all_scenarios, 'RfR Total Costs')
combine_columns(policies_all_scenarios, 'Expected Evacuation Costs')

Generate statistics

In [ ]:
aggregated_outcomes = pd.concat([experiments[['scenario','policy']],policies_all_scenarios.iloc[:,-5:]], axis=1)

stats_df = pd.DataFrame()
for column in aggregated_outcomes.iloc[:,2:].columns:
    stats_df[column + ' mean'] = aggregated_outcomes.groupby('policy').agg({column:['mean']})
    stats_df[column + ' std'] = aggregated_outcomes.groupby('policy').agg({column:['std']})

with open(r'../generated_datasets/policy_uncertainty_aggregated.pkl', 'wb') as file_pi:
    pickle.dump(aggregated_outcomes, file_pi)

### Gelderland - Overijssel Split

In [ ]:
def combine_columns_province(dataframe, name):
    combined_columns_gelderland = []
    combined_columns_overijssel = []
    for x in dataframe.columns:
        if name in x:
            if x.startswith('A.1') or x.startswith('A.2') or x.startswith('A.3'):
                combined_columns_gelderland.append(x)
            if x.startswith('A.4') or x.startswith('A.5'):
                combined_columns_overijssel.append(x)

    dataframe['Total ' + str(name) + ' Gelderland'] = dataframe[combined_columns_gelderland].sum(axis=1)
    dataframe['Total ' + str(name) + ' Overijssel'] = dataframe[combined_columns_overijssel].sum(axis=1)

In [ ]:
provinces = policies_all_scenarios.copy()

combine_columns_province(provinces, 'Expected Annual Damage')
combine_columns_province(provinces, 'Dike Investment Costs')
combine_columns_province(provinces, 'Expected Number of Deaths')
combine_columns_province(provinces, 'RfR Total Costs')
combine_columns_province(provinces, 'Expected Evacuation Costs')

In [ ]:
aggregated_outcomes_provinces = pd.concat([experiments[['scenario','policy']],provinces.iloc[:,-10:]], axis=1)

with open(r'../generated_datasets/policy_uncertainty_provinces.pkl', 'wb') as file_pi:
    pickle.dump(aggregated_outcomes_provinces, file_pi)